# Feature Engineering

Now that I have an idea about past power usage, and common patterns, and the relationships among meter readings and `Global_active_power` from the EDA notebook, I am ready to move on to feature engineering: transforming this into a full training dataset for an ML model.

Feature engineering is all about working with existing data to create helpful variables for your predictive goal. In the most ideal case, you will not need to do much of this at all, but it is often a really important step in creating the best, input features for prediction. 

In the rest of this notebook, I will go through the following steps: 
>1. **Loading the data**: Loading the data in as a DataFrame
2. **Cleaning the data**: Getting clean data by using code I created in the EDA notebook for dealing with nan's + re-sampling
3. **Train/test split**: Splitting the clean data into train/test sets
4. **Creating features**: Transforming data and creating new input features that I think will have predictive power
5. **Saving data**: Saving my transformed data so I can later load it for model training! (And un-mounting Trove data.)

Just like how exploring data was an iterative process, so is feature engineering. 

I often find it useful to start with just a few basic features, use these for training an ML/DL model, and only return to create more features and add complexity, once I've learned from those initial training runs.


Let's start by loading in the usual resources.

In [1]:
# basic data viz and processing libraries
import turitrove as trove
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


!! Could not derive your terminal width !!


---
# Load (or Mount) the Data

In the last notebook, I mounted some battery charging Trove data. 

>I still have it mounted and so I can access it as I would any file on my computer: by path, which I can always find and copy in Finder > Get Info (for a file) > Where (Copy location).

If I unmounted the data, I can mount it again, and proceed as usual; reading in a file as a DataFrame. 

In other cases, I may have a temporary local file of data that I can read directly with pandas `.read_pickle()` or `read_csv()`.

In [2]:
TROVE_URI = 'dataset/household_power_consumption@1.0.0'
trove.umount(TROVE_URI)

# TODO: un-comment to define your own path OR mount data from Trove and get path
# data_path = '/Users/me/Deep_Learning/...household_power_consumption-1.0.0/data/raw/power_consumption.txt'

# create local temp_data dir and mount data there
if not os.path.isdir('temp_data'):
    os.makedirs('temp_data')
    
power_dataset = trove.mount(TROVE_URI, 'temp_data')

data_path = power_dataset.raw_file_path + '/'+ power_dataset.primary_index['path'][0]


# this is semi-colon separated, so let's use that to read this in as a DataFrame
sep = ';'
na_vals = ['nan', '?']

df = pd.read_csv(data_path, sep=sep, na_values = na_vals,
                 dtype={'Global_active_power': float, 
                        'Voltage': float,
                        'Sub_metering_1': float,
                        'Sub_metering_2': float,
                        'Sub_metering_3': float
                       })


dataset/household_power_consumption@1.0.0 is umounted from /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/data/table
dataset/household_power_consumption@1.0.0 is umounted from /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/data/raw
folder /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/data is removed
terms.md is removed from /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/terms.md
readme.md is removed from /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/readme.md
info.json is removed from /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/info.json
Directory /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_

N/A% (0 of 100) |                                        |Elapsed Time: 0:00:00


Mount succeeded
mount in progress: /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/data/raw
96.9MB disk space allocated for caching /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/data/raw


100% (100 of 100) |######################################|Elapsed Time: 0:00:00


Mount succeeded
dataset/household_power_consumption@1.0.0 is mounted at /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data


---
# Clean the data
I also have code from the exploratory data analysis notebook that I am porting over here. This code does two things:
1. Cleaning the data by imputing null values with mean values
2. Re-sampling the data by hour instead of minute, which will be useful for hour-by-hour predictions


In [3]:
##-----   Code to deal with nan values  -----##

# helper function to fill NaN values with a column average
def fill_nan_with_mean(df):
    '''Fills NaN values in a given dataframe with the average values in a column.
    
       :param df: a DataFrame that is assumed to contain cols of float values
       :return: returns a DataFrame with all null values replaced with the mean val for a column'''
    
    # filling nan with mean value of any columns
    for col in list(df.columns.values):  
        # ignoring date/time cols
        if col != 'Date' and col != 'Time':
            df[col].iloc[:]=df[col].iloc[:].fillna(df[col].iloc[:].mean())
        
    return df


# Note: going to ignore SettingWithCopyWarning for now
clean_power_df = fill_nan_with_mean(df)


##-----   Re-sample by hourly mean for hourly predictions  -----##

# indexing by date-time for explorations
format_dates = '%d/%m/%Y %H:%M:%S'

# create one date-time index, converted to correct type
clean_power_df['Date-Time'] = pd.to_datetime(clean_power_df['Date']+' '+clean_power_df['Time'], format=format_dates)
#clean_power_df.drop(['Date', 'Time']) # drop old cols that were merged
clean_power_df = clean_power_df.set_index(pd.DatetimeIndex(clean_power_df['Date-Time']))


# resample every hour
freq = 'H'
# calculate the mean active power for a day
hourly_power_df = clean_power_df.resample(freq).mean()


# print some stats about the data
print('Data shape: ', hourly_power_df.shape)
hourly_power_df.head()


/Users/cezannecamacho/opt/anaconda3/envs/deep-learning/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Data shape:  (34589, 5)


,Global_active_power,Voltage,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date-Time,,,,,
2006-12-16 17:00:00,4.222889,234.643889,0.0,0.527778,16.861111
2006-12-16 18:00:00,3.632200,234.580167,0.0,6.716667,16.866667
2006-12-16 19:00:00,3.400233,233.232500,0.0,1.433333,16.683333
2006-12-16 20:00:00,3.268567,234.071500,0.0,0.000000,16.783333
2006-12-16 21:00:00,3.056467,237.158667,0.0,0.416667,17.216667


---
# Train/Test Split

I'll evaluate eventual models I train on a _test_ set of data. 

For machine learning tasks like classification or regression, I typically create train/test data by _randomly_ splitting examples into different sets. 
> However, for time forecasting it's important to do this train/test split in **time** rather than by random, individual data points. 

As a design decision, I am going to truncate by year; leaving the 2010 data as our test data and the years prior as our training data. There are several other ways you might approach this split that would work, too. 

### Splitting early to avoid data leakage
I typically do this split early on so as to avoid leakage between any engineered features I calculate, which should only come from the training data. 

This is because, in reality, I will not have access to any future (test) data when making my calculations and using test data to do any data transformation would constitute a [**data leakage**](https://en.wikipedia.org/wiki/Leakage_(machine_learning)), which is when information from outside the training dataset is used in creating a model (or in creating data that a model learns from ). 

### Split size

You typically want a 20/80 or 30/70 test/train split so that you have enough, representative data in your test set to do a meaningful evaluation. For larger datasets, at Apple, we may take more or less data depending on our representation goals in that data. 

In [4]:
# filtering years <2010 for train data, == 2010 for test data
train_hourly_power = hourly_power_df[hourly_power_df.index.year < 2010]
test_hourly_power = hourly_power_df[hourly_power_df.index.year == 2010]

# check that test is during 2010
test_hourly_power.head()

,Global_active_power,Voltage,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date-Time,,,,,
2010-01-01 00:00:00,1.085867,241.300000,0.0,0.00,8.516667
2010-01-01 01:00:00,0.651233,241.620667,0.0,0.80,0.650000
2010-01-01 02:00:00,0.634600,244.012333,0.0,0.00,0.666667
2010-01-01 03:00:00,0.653000,244.063833,0.0,0.00,1.383333
2010-01-01 04:00:00,0.646067,241.649333,0.0,0.75,0.666667


In [5]:
## Check that I have a reasonable amount of test data, relative to train
## should be between 20-30%

#print('Total data len: ', len(hourly_power_df))
print('Train len: ', len(train_hourly_power))
print('Test len: ', len(test_hourly_power))
print()
print('Amount of test data as % of total data: ', 
      str(len(test_hourly_power)/(len(test_hourly_power)+len(train_hourly_power))) )


Train len:  26671
Test len:  7918

Amount of test data as % of total data:  0.22891670762381103


---
# Transforming Data and Creating Features

Creating features can sometimes happen before data cleaning and splitting; it is only important to split first if you are using past data (training) to create features for future data (test). 

At this point, I am again going to revisit some of my problem-framing notes; these notes are threaded throughout the rest of my explorations and data transformation steps. 

Recall that my **predictive goal** is to predict future, global, power usage on an hourly-basis (e.g. for 9am, 10am, etc.) from past usage data. Next, I'll add some specificity to my hypothesis, so that I can actually try it out:

>**Hypothesis**: I suspect real-time power meter readings and _past_ hourly usage—specifically the mean or median global active power around a specific hour—will be great predictors for an ML model. 


### Creating statistical features

* Take the mean around each hr from the training data
* Add the mean as a column in train and test data, 'Mean_hourly_power'
* 📍Later: could try taking a median or a longer, rolling average, or bucketed average for usage every 2, 4, 8 hrs; std dev or other statistical measures may also be useful.

To get a mean (and std dev.) around each hour, I am first going to create empty python lists to add these values to. Then, in the below code, I am iterating over each hour (0 through 23) by using a for loop: `for hr in range(24)`.
* I grab the selected `hr`'s worth of data by filtering for that hr's timestamp in my training data
* Then I use numpy's `mean` and `std` functions to do my calculations on that hour's `Gloval_active_power` data
* Finally I append these values to my lists

I then check to see if each of the means for every hour in a day seem reasonable.

In [7]:
mean_usage_by_hr = []

std_dev_by_hr = []

for hr in range(24):
    hour_group = train_hourly_power[(train_hourly_power.index.hour == hr)]
    # calculating stats
    avg_hourly_power = np.mean(hour_group['Global_active_power'])
    std_hourly_power = np.std(hour_group['Global_active_power'])
    mean_usage_by_hr.append(avg_hourly_power)
    std_dev_by_hr.append(std_hourly_power)


In [11]:
mean_usage_by_hr

[0.6575444501391943,
 0.5339421754146448,
 0.47089431485413635,
 0.4440237235694603,
 0.4468450083745834,
 0.460485603290283,
 0.810142608990853,
 1.5038341425055168,
 1.4821339302288141,
 1.3323408921885123,
 1.251101645601084,
 1.2302188673232564,
 1.1907100548792338,
 1.1329363790605569,
 1.0729514869027257,
 0.9832678986796684,
 0.9578782471875144,
 1.0728405075275789,
 1.352090364123325,
 1.7885808356723576,
 1.9529222074931454,
 1.930303706519472,
 1.4382587367208581,
 0.9111828178771005]

### Apply and lambda

Then I can take advantage of the index locations of these values—at index 0 I have the mean `Global_active_power` for hour 0, at index 1 I have the mean for hour 1, and so on. And I add these hourly-means as a **new column** `Mean_hourly_power` to my training and test data based on the time stamp of a given row of data. 

`train_hourly_power['Mean_hourly_power'] = train_hourly_power['Date-Time'].apply(lambda x: mean_usage_by_hr[x.hour])`

You'll notice the usage of pandas filtering `.apply()` and python `lambda` functions. 
* `.apply()` allows you to apply a function to all the rows in a given column or DataFrame that comes before the `.apply()`
* `lambda` gives you a way to define a function in-line; in this case the `x` refers to the data that comes before `.apply()` and the function applied to each row is what follows `lambda x: ` —in this case a simple function gets the mean values from a list of means using the `x.hour` as an index

I use these all the time in data-processing work, but it's taken quite a while to get used to this syntax. 

>If you're interested in learning more, I recommend this [blog post on apply + lambda functions](https://towardsdatascience.com/apply-and-lambda-usage-in-pandas-b13a1ea037f7). 

In [12]:
# reset index so we can access date time value in a column and use .apply() in next cell
train_hourly_power = train_hourly_power.reset_index()
test_hourly_power = test_hourly_power.reset_index()

In [13]:
train_hourly_power.head()

,Date-Time,Global_active_power,Voltage,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16 17:00:00,4.222889,234.643889,0.0,0.527778,16.861111
1,2006-12-16 18:00:00,3.632200,234.580167,0.0,6.716667,16.866667
2,2006-12-16 19:00:00,3.400233,233.232500,0.0,1.433333,16.683333
3,2006-12-16 20:00:00,3.268567,234.071500,0.0,0.000000,16.783333
4,2006-12-16 21:00:00,3.056467,237.158667,0.0,0.416667,17.216667


In [14]:
train_hourly_power['Mean_hourly_power'] = train_hourly_power['Date-Time'].apply(
    lambda x: mean_usage_by_hr[x.hour])

test_hourly_power['Mean_hourly_power'] = test_hourly_power['Date-Time'].apply(
    lambda x: mean_usage_by_hr[x.hour])


In [15]:
train_hourly_power['Std_hourly_power'] = train_hourly_power['Date-Time'].apply(lambda x: std_dev_by_hr[x.hour])
test_hourly_power['Std_hourly_power'] = test_hourly_power['Date-Time'].apply(lambda x: std_dev_by_hr[x.hour])

train_hourly_power.head()

,Date-Time,Global_active_power,Voltage,Sub_metering_1,Sub_metering_2,Sub_metering_3,Mean_hourly_power,Std_hourly_power
0,2006-12-16 17:00:00,4.222889,234.643889,0.0,0.527778,16.861111,1.072841,0.934144
1,2006-12-16 18:00:00,3.632200,234.580167,0.0,6.716667,16.866667,1.352090,1.058081
2,2006-12-16 19:00:00,3.400233,233.232500,0.0,1.433333,16.683333,1.788581,1.151857
3,2006-12-16 20:00:00,3.268567,234.071500,0.0,0.000000,16.783333,1.952922,1.173999
4,2006-12-16 21:00:00,3.056467,237.158667,0.0,0.416667,17.216667,1.930304,1.100202


### Date-time features

In [16]:
# also grab starting hr (0-24) in a column 
train_hourly_power['Start_hr'] = train_hourly_power['Date-Time'].apply(lambda x: x.hour)
test_hourly_power['Start_hr'] = test_hourly_power['Date-Time'].apply(lambda x: x.hour)

train_hourly_power.head()


,Date-Time,Global_active_power,Voltage,Sub_metering_1,Sub_metering_2,Sub_metering_3,Mean_hourly_power,Std_hourly_power,Start_hr
0,2006-12-16 17:00:00,4.222889,234.643889,0.0,0.527778,16.861111,1.072841,0.934144,17
1,2006-12-16 18:00:00,3.632200,234.580167,0.0,6.716667,16.866667,1.352090,1.058081,18
2,2006-12-16 19:00:00,3.400233,233.232500,0.0,1.433333,16.683333,1.788581,1.151857,19
3,2006-12-16 20:00:00,3.268567,234.071500,0.0,0.000000,16.783333,1.952922,1.173999,20
4,2006-12-16 21:00:00,3.056467,237.158667,0.0,0.416667,17.216667,1.930304,1.100202,21


---
# Save Transformed Data
I cleaned and added useful columns to this dataset. The next step is to save my work; I typically save data as either csv files or in binary, pickle format (which is much faster for loading and saving for large datasets and preserves your formatting)!

> Below, I am defining the order of my columns—the target var typically goes last, and date-time is typically dropped entirely or included as the first column. 

And I am saving separate train/test files using `.to_pickle()` which saves these in a binary format that can be shared with my team, and used for training ML models later on (after reading in again with `read_pickle()`.

In [17]:
# date-time column is dropped 
train_hourly_power = train_hourly_power.drop(['Date-Time'], axis=1)
test_hourly_power = test_hourly_power.drop(['Date-Time'], axis=1)


In [18]:
# target var is last, by convention

# re-index with Global_active_power as the last column
column_order = ['Mean_hourly_power', 'Std_hourly_power', 'Start_hr', 'Voltage', 
                'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3','Global_active_power']

train_hourly_power = train_hourly_power.reindex(columns=column_order)
test_hourly_power = test_hourly_power.reindex(columns=column_order)

In [19]:
# Save as pkl format in data dir
## I created the data/ dir in the same directory as this notebook
if not os.path.isdir('data'):
    os.makedirs('data')
    
train_hourly_power.to_pickle('data/train_hourly.pkl')
test_hourly_power.to_pickle('data/test_hourly.pkl')

### Unmount Trove data

Now that I have my saved, transformed data stored locally—which I will delete after all my ML model experiments—I can un-mount my mounted Trove dataset because I no longer need access to this original volume!

In [20]:
## Before you go: un-mounting Trove dataset since now I have locally-stored, transformed csv's to work with
trove.umount(TROVE_URI)


dataset/household_power_consumption@1.0.0 is umounted from /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/data/table
dataset/household_power_consumption@1.0.0 is umounted from /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/data/raw
folder /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/data is removed
terms.md is removed from /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/terms.md
readme.md is removed from /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/readme.md
info.json is removed from /Users/cezannecamacho/Desktop/Deep_Learning/DL_Notebooks_recording/temp_data/household_power_consumption-1.0.0/info.json
